<h1 style="text-align: center;"> LOCUS COERULEUS (LC) - MEAN INTENSITY EXTRACTION </h1>

This notebook continues extraction for participants missed during first extraction, due to server crash. 

In [ ]:
import os
import pandas as pd
import subprocess

In [ ]:
# defining base directory and LC mask names
base_dir = "/hus/home/oliwin/data/study_name"

LC_mask = f"{base_dir}/LC_mask.nii.gz"
mask_lc_l = f"{base_dir}/LC_L_mask.nii.gz"
mask_lc_r = f"{base_dir}/LC_R_mask.nii.gz"

In [ ]:
# 4th list of missing participants from first extraction
# list contains 12 of the missing participants from the first extraction

missing_patients = ["00008B22", "00008C1B", "00008E89", "00008EA0", "000090BE", 
 "000090DC", "0000991B", "00009D8F", "0000AA56", "0000AF0D", 
 "0000B5CD", "0000C2A9"]

# dictionary to store participant-wise data
patients_data = {}

# defining missing list nr 4
output_file = 'mri_data_LC_missing_patients_4.csv'

header = ['Patient Number']
all_keys = set()

# saving current results to csv
def save_progress():
    global header

    all_keys.update(*(intensities.keys() for intensities in patients_data.values()))
    header = ['Patient Number'] + sorted(all_keys)

    output_data = []
    for patient_number, intensities in patients_data.items():
        row = [patient_number]
        for col in header[1:]:
            row.append(intensities.get(col, None))  # use None for missing values
        output_data.append(row)

    output_df = pd.DataFrame(output_data, columns=header)
    output_df.to_csv(output_file, index=False)
    print(f"Progress saved to {output_file}")


try:
    # loop through subdirectories 'FSL' in base_dir (recursively)
    for root, dirs, files in os.walk(base_dir):
        if "FSL" in dirs:
            fsl_dir = os.path.join(root, "FSL")
            
            # extracting participant number from folder name 
            patient_number = os.path.basename(root)

            # processing only the specified missing participants
            if patient_number not in missing_patients:
                continue

            # loop through NIfTI files in the FSL directory
            for filename in os.listdir(fsl_dir):
                # process only files with "mni", "nii.gz"
                if (
                    "mni" in filename
                    and "mask" not in filename
                    and "new" not in filename
                    and "flirt" not in filename
                    and "flirt_masked" not in filename
                    and not filename.endswith(".mat")
                    and filename.endswith(".nii.gz")
                ):
                    input_file = os.path.join(fsl_dir, filename)

                    print(f"Processing file: {input_file}")

                    # loop through the LC masks (left, right, combined) - each MRI file is processed three times
                    for mask_file in [LC_mask, mask_lc_r, mask_lc_l]:
                        try:
                            # use FLIRT to perform registration of the LC masks
                            flirt_output = f"{input_file}_flirt_{os.path.basename(mask_file).split('.')[0]}"
                            flirt_matrix = f"{flirt_output}.mat"
                            flirt_command = f"flirt -in {input_file} -ref {mask_file} -out {flirt_output} -omat {flirt_matrix}"
                            process_flirt = subprocess.run(flirt_command, shell=True, capture_output=True, text=True)

                            if process_flirt.returncode != 0:
                                print(f"Error applying FLIRT to {filename} with mask {mask_file}: {process_flirt.stderr}")
                                continue  # continue with the next mask for same file

                            # use fslmaths to apply the mask and threshold negative values to zero
                            flirt_masked = f"{flirt_output}_masked"
                            fslmaths_command = f"fslmaths {flirt_output} -mas {mask_file} -thr 0 {flirt_masked}"
                            process_fslmaths = subprocess.run(fslmaths_command, shell=True, capture_output=True, text=True)

                            if process_fslmaths.returncode != 0:
                                print(f"Error applying fslmaths to {filename} with mask {mask_file}: {process_fslmaths.stderr}")
                                continue  # continue with the next mask for same file

                            # use fslstats to extract mean intensity within the region
                            stats_command = f"fslstats {flirt_masked} -M"  # -M for Mean Intensity 
                            process_stats = subprocess.run(stats_command, shell=True, capture_output=True, text=True)

                            if process_stats.returncode != 0:
                                print(f"Error processing {filename}: {process_stats.stderr}")
                                continue  # continue with the next mask for same file

                            
                            stats = process_stats.stdout.strip().split()
                            mean_intensity = float(stats[0])  

                            # remove intermediate files and temporary NIfTI outputs
                            try:
                                for file_to_delete in [flirt_matrix, flirt_output + ".nii.gz", flirt_masked + ".nii.gz"]:
                                    if os.path.exists(file_to_delete):
                                        os.remove(file_to_delete)
                                    else:
                                        print(f"File not found: {file_to_delete}")
                            except OSError as e:
                                print(f"Error deleting files for {filename}: {e}")

                            # add the data to participants's entry
                            if patient_number not in patients_data:
                                patients_data[patient_number] = {}

                            mask_name = os.path.basename(mask_file).split('.')[0]  
                            filename_base = filename.replace('.nii.gz', '')  
                            mask_key = f'{mask_name}_{filename_base}'

                            patients_data[patient_number][mask_key] = mean_intensity

                            # saving mean intensity immediately after each file is processed with all three masks
                            save_progress()

                        except Exception as e:
                            print(f"Error processing {filename} with mask {mask_file} for patient {patient_number}: {e}")
                            continue  # continue to next mask or next file

    # final save after all participants and files have been processed
    print("Final saving progress...")
    save_progress()

# handle interruptions and unexpected errors
except KeyboardInterrupt:
    print("Process interrupted, saving progress...")
    save_progress()

except Exception as e:
    print(f"An error occurred: {e}")
    save_progress()

Processing file: /hus/home/oliwin/data/study_name/0000AA56/FSL/STAGE_tSWI_mIP_ECHO-3_e3_reg_mni.nii.gz
Progress saved to mri_data_LC_missing_patients_4.csv
Progress saved to mri_data_LC_missing_patients_4.csv
Progress saved to mri_data_LC_missing_patients_4.csv
Processing file: /hus/home/oliwin/data/study_name/0000AA56/FSL/STAGE_tSWIhpf_ECHO-3_e3_reg_mni.nii.gz
Progress saved to mri_data_LC_missing_patients_4.csv
Progress saved to mri_data_LC_missing_patients_4.csv
Progress saved to mri_data_LC_missing_patients_4.csv
Processing file: /hus/home/oliwin/data/study_name/0000AA56/FSL/mprage_reg_mni.nii.gz
Progress saved to mri_data_LC_missing_patients_4.csv
Progress saved to mri_data_LC_missing_patients_4.csv
Progress saved to mri_data_LC_missing_patients_4.csv
Processing file: /hus/home/oliwin/data/study_name/0000AA56/FSL/STAGE_CROWN_PD_MAP_reg_mni.nii.gz
Progress saved to mri_data_LC_missing_patients_4.csv
Progress saved to mri_data_LC_missing_patients_4.csv
Progress saved to mri_data_LC_